## Détection d'objets

### Préambule

Le dossier racine doit contenir:  
-ce notebook  
-le dossier ImageAI  
-le dossier Cascade tel qu'il est donné dans le TP6 (donc avec ses propres réseaux)  
-le réseau de neurone pré-entrainé pour ImageAI  
-la vidéo road.mp4  
-2 dossiers pré-créés (path, et detectedpath) qui conserveront les images durant l'execution.  
  
Afin de ne pas surcharger la taille du projet lors du rendu, seuls ce notebook et les dossiers d'images (pleins) seront rendus (afin de conserver une trace d'une execution fonctionnelle et de ne pas perdre du temps à executer ce notebook pendant une heure.


### Cœur du projet

On commence par importer ImageAI (assez lourde).

In [ ]:
import os
import sys
#Chemin de ce dossier (à changer)
project_path= os.path.abspath("/Python/PI_TP7_Detection_objets/")
sys.path.append(project_path)

# Mettre le chemin où vous avez placé la bibliothèque ImageAI
imageai_path = os.path.join(project_path,"ImageAI") 
sys.path.append(imageai_path)
from imageai.Detection import ObjectDetection
import cv2

Puis on importe les dépendances de Cascade-LC.

In [3]:
import torch
import torchvision
import numpy as np
import random
import math

# Pour charger les données et visualiser les résultats
from PIL import Image
from torchvision.transforms import ToTensor, ToPILImage
from matplotlib.pyplot import imshow, figure, subplots

# Pour charger les réseaux neuronaux
from Cascade.models.erfnet import Net as ERFNet
from Cascade.models.lcnet import Net as LCNet


from Cascade.functions import color_lanes


if torch.cuda.is_available():
    map_location=lambda storage, loc: storage.cuda()
else:
    map_location='cpu'

On définit quelques variables globales afin de pouvoir executer les cellules séparément:

In [4]:
#Les différents dossiers devront être créé avant l'execution.
#Les images de la vidéo originales seront stockées dans le dossier:
path = os.path.join(project_path,"CapturedFrames")

#Les images finales (avec les objets détectés) sont envoyées dans le dossier :
detectedpath = os.path.join(project_path,"DetectedFrames")

# On ne traite pas toutes les images, donc on ne traitera que une image sur :
frames_step = 20


execution_path = os.getcwd()

In [5]:
#Variables propres à Cascade-LC
DESCRIPTOR_SIZE = 64
NUM_CLASSES_SEGMENTATION = 5
NUM_CLASSES_CLASSIFICATION = 3

#Chemin vers le réseau classifieur
model_path = 'Cascade/pretrained/classification_{}_{}class.pth'.format(DESCRIPTOR_SIZE, NUM_CLASSES_CLASSIFICATION)
#Chemin vers le réseau de segmentation
seg_path = 'Cascade/pretrained/erfnet_tusimple.pth'

On ouvre la vidéo et on calcule le nombre d'images que l'on traitera (todo_frames):

In [6]:
# On ouvre la vidéo
movie= cv2.VideoCapture(os.path.join(project_path, "road.mp4"))

total_frames = movie.get(cv2.CAP_PROP_FRAME_COUNT)
todo_frames = int(total_frames // frames_step)
print(todo_frames)

235


On commence en extrayant les frames de la vidéo que l'on souhaite traiter.  
ATTENTION : Si le dossier path a déjà été rempli, il n'est pas obligatoire de le refaire.

In [7]:
for i in range(todo_frames):
    movie.set(1, i * frames_step)
    success, frame = movie.read()  

    cv2.imwrite(os.path.join(path , 'img' + str(i) + '.jpg'), frame)
 
movie.release()

In [8]:
#On récupère une des images capturée afin de récupérer les dimensions de la vidéo.
imageTest=cv2.imread(os.path.join(path , 'img'+str(0)+'.jpg'))
height,width,layers=imageTest.shape

On défini un détecteur d'objets ImageAI et on lui donne le réseau de neurone pré-entrainé.

In [9]:
detector = ObjectDetection()
detector.setModelTypeAsYOLOv3()

#On ajoute ici notre réseau pré-entrainé
detector.setModelPath( os.path.join(execution_path , "pretrained-yolov3.h5"))
detector.loadModel()

Puis on charge le réseau de Cascade-LC avec pytorch.

In [10]:
# Création des reseaux Cascade et chargement des poids
segmentation_network = ERFNet(NUM_CLASSES_SEGMENTATION)
classification_network = LCNet(NUM_CLASSES_CLASSIFICATION, DESCRIPTOR_SIZE, DESCRIPTOR_SIZE)

segmentation_network.load_state_dict(torch.load(seg_path, map_location = map_location))
classification_network.load_state_dict(torch.load(model_path, map_location = map_location))

segmentation_network = segmentation_network.eval()
classification_network = classification_network.eval()

if torch.cuda.is_available():
    segmentation_network = segmentation_network.cuda()
    classification_network = classification_network.cuda()

On ajoute la fonction d'extraction des descripteurs (légèrement modifiée lors de l'initialisation de single_lane) qu'on appelera pour chaque image préalablement segmentées.

In [11]:
def extract_descriptors(label, image):
    # avoids problems in the sampling
    eps = 0.01
    
    # The labels indices are not contiguous e.g. we can have index 1, 2, and 4 in an image
    # For this reason, we should construct the descriptor array sequentially
    inputs = torch.zeros(0, 3, DESCRIPTOR_SIZE, DESCRIPTOR_SIZE)
    if torch.cuda.is_available():
        inputs = inputs.cuda()
        
    # This is needed to keep track of the lane we are classifying
    mapper = {}
    classifier_index = 0
    
    # Iterating over all the possible lanes ids
    for i in range(1, NUM_CLASSES_SEGMENTATION):
        # This extracts all the points belonging to a lane with id = i
        #Cette ligne a été modifié car elle ne permettait pas à la fonction d'être appelée plus de 128 fois.
        single_lane = label.eq(i).view(-1).nonzero(as_tuple=False).squeeze(1)
        
        # As they could be not continuous, skip the ones that have no points
        if single_lane.numel() == 0:
            continue
        
        # Points to sample to fill a squared desciptor
        sample = torch.zeros(DESCRIPTOR_SIZE * DESCRIPTOR_SIZE)
        if torch.cuda.is_available():
            sample = sample.cuda()
            
        sample = sample.uniform_(0, single_lane.size()[0] - eps).long()
        sample, _ = sample.sort()
        
        # These are the points of the lane to select
        points = torch.index_select(single_lane, 0, sample)
        
        # First, we view the image as a set of ordered points
        descriptor = image.squeeze().view(3, -1)
        
        # Then we select only the previously extracted values
        descriptor = torch.index_select(descriptor, 1, points)
        
        # Reshape to get the actual image
        descriptor = descriptor.view(3, DESCRIPTOR_SIZE, DESCRIPTOR_SIZE)
        descriptor = descriptor.unsqueeze(0)
        
        # Concatenate to get a batch that can be processed from the other network
        inputs = torch.cat((inputs, descriptor), 0)
        
        # Track the indices
        mapper[classifier_index] = i
        classifier_index += 1
        
    return inputs, mapper


La fonction Cascade sera appelée dès qu'on souhaitera trouver des lignes dans une image, et coloriera cette dernière en conséquence.

In [12]:
#La fonction blend() originale n'était pas compatible avec des images ouvertes par OpenCV
def myOwnBlend(img,classificator):
    height, width, colors = img.shape
    for i in range(height):
        for j in range(width):
            for c in range(colors):
                if classificator[i][j][c]:
                    img[i][j][c] = classificator[i][j][c]
    return img

#Classification des lignes pour une image ouverte avec openCV
def Cascade(im):
    im_tensor = ToTensor()(im)
    im_tensor = im_tensor.unsqueeze(0)
    
    out_segmentation = segmentation_network(im_tensor)
    out_segmentation = out_segmentation.max(dim=1)[1]

    descriptors, index_map = extract_descriptors(out_segmentation, im_tensor)
    #On récupère les différentes classes(lignes différentes) grâce aux descripteurs.
    classes = classification_network(descriptors).max(1)[1]
    
    #Puis on colorie l'image suivant les lignes que l'on trouve
    out_segmentation_np = out_segmentation.cpu().numpy()[0]
    
    classification_picture = np.zeros((height,width, 3))
    for i, lane_index in index_map.items():
        if classes[i] == 0: # Continuous
            color = (255, 0, 0)
        elif classes[i] == 1: # Dashed
            color = (0, 255, 0)
        elif classes[i] == 2: # Double-dashed
            color = (0, 0, 255)
        else:
            raise
        classification_picture[out_segmentation_np == lane_index] = color
    #image finale
    finale = myOwnBlend(im, classification_picture)
    return finale

On applique le détecteur d'objets et le réseau en cascade sur chaque image afin de reconnaitre des objets connus de notre réseau.  
ATTENTION : Ce processus risque de prendre énormément de temps (17 secondes par image * todoframes => une heure), à ne faire que si le dossier detectedpath est vide.

In [13]:
for j in range(todo_frames):
    #Detection d'objets (une ligne).
    tmp_image,ret = detector.detectObjectsFromImage(input_image = os.path.join(path , 'img' + str(j) + '.jpg'), 
                                    minimum_percentage_probability = 40,
                                    output_type = 'array')
    #Detection des lignes dans un second temps
    finale_image = Cascade(tmp_image)
    cv2.imwrite(os.path.join(detectedpath,'img' + str(j) + '.jpg'), finale_image)
    

On compile maintenant les nouvelles images avec les objets détectés sous la forme d'une vidéo.

In [14]:
#Le 3eme paramètre est le framerate de la vidéo,vous pouvez l'augmenter si vous voulez que la vidéo soit plus rapide.
#J'ai executé ce code sur MacOS, il est donc probable qu'il faille changer le codec fourcc par (*'XVID')
video=cv2.VideoWriter('vehicles_detection.avi',cv2.VideoWriter_fourcc(*'DIVX'), 5,(width,height))

In [15]:
#On écrit les images une par une dans la vidéo
for k in range(todo_frames):
    image=cv2.imread(os.path.join(detectedpath , 'img'+str(k)+'.jpg'))
    video.write(image)

video.release()

Notes :  
On remarque vers les 3/5ièmes de la vidéo finale que les piétons sont correctement identifiés comme des personnes (on ne peut les voir que quelques images). La liste des objets reconnaissables est donnée dans le fichier _init_.py du dossier Detection de ImageAI.  
Il arrive sur certaines images que l'avant de la voiture qui filme soit détecté. Il arrive aussi que certaines voitures lointaines soient reconnues en tant que camions ("trucks") ou en vaches, la faible résolution diminue évidemment la précision du réseau dans ce genre de cas.  
Des maisons sont aussi reconnues comme étant des camions, ou des panneaux de directions comme des feux tricolores. Ce genre de mauvaise classifications est souvent considérée comme imprécise (moins de 75% de certitude).   
La détection de lignes est assez imprécise par rapport à l'autre réseau, surtout quand il ne s'agit pas de lignes continues.